In [128]:
from bs4 import BeautifulSoup
import re
import csv

In [11]:
with open('tomo1-vocab-kindle.html', encoding='utf-8') as f:
    html_file = f.read()


In [12]:
html_file

'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "XHTML1-s.dtd" >\n<html xmlns="http://www.w3.org/TR/1999/REC-html-in-xml" xml:lang="en" lang="en">\n\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"></meta>\n<style>\n.bodyContainer {\n    font-family: Arial, Helvetica, sans-serif;\n    text-align: center;\n    padding-left: 32px;\n    padding-right: 32px;\n}\n\n.notebookFor {\n    font-size: 18px;\n    font-weight: 700;\n    text-align: center;\n    color: rgb(119, 119, 119);\n    margin: 24px 0px 0px;\n    padding: 0px;\n}\n\n.bookTitle {\n    font-size: 32px;\n    font-weight: 700;\n    text-align: center;\n    color: #333333;\n    margin-top: 22px;\n    padding: 0px;\n}\n\n.authors {\n    font-size: 13px;\n    font-weight: 700;\n    text-align: center;\n    color: rgb(119, 119, 119);\n    margin-top: 22px;\n    margin-bottom: 24px; \n    padding: 0px;\n}\n\n.sectionHeading {\n    font-size: 24px;\n    font-w

In [15]:
soup = BeautifulSoup(html_file, 'lxml')

C:\Users\sam\AppData\Local\Temp\ipykernel_19216\2660174346.py:1: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_file, 'lxml')


In [22]:
note_text_divs = soup.find_all("div", class_="noteText")
note_text_divs

[<div class="noteText">restriñimientos.
 <h3 class="noteHeading">Highlight (<span class="highlight_blue">blue</span>) - —I— &gt; Location 598</h3></div>,
 <div class="noteText">sucedáneos.
 <h3 class="noteHeading">Highlight (<span class="highlight_blue">blue</span>) - —I— &gt; Location 602</h3></div>,
 <div class="noteText">remordimientos
 <h3 class="noteHeading">Highlight (<span class="highlight_blue">blue</span>) - —I— &gt; Location 609</h3></div>,
 <div class="noteText">mentecato.
 <h3 class="noteHeading">Highlight (<span class="highlight_blue">blue</span>) - —I— &gt; Location 611</h3></div>,
 <div class="noteText">veleidoso,
 <h3 class="noteHeading">Highlight (<span class="highlight_blue">blue</span>) - —I— &gt; Location 611</h3></div>,
 <div class="noteText">aturdirse,
 <h3 class="noteHeading">Highlight (<span class="highlight_blue">blue</span>) - —I— &gt; Location 626</h3></div>,
 <div class="noteText">desvencijada,
 <h3 class="noteHeading">Highlight (<span class="highlight_blue"

### Get all the words from the annotations

In [82]:
words = []
for div in note_text_divs:
    # can also be a phrase
    word = div.contents[0].strip('\n')
    words.append(word)

### Get the chapter info for the annotated words

In [120]:
def get_chapter_numeral(div_string):
    numeral_re = r'[IXV]+'
    ch_numeral_search = re.search(numeral_re, div_string)
    try:
        assert ch_numeral_search is not None
    except AssertionError:
        print(div_string)
    start, end = ch_numeral_search.span()
    return div_string[start:end]


In [121]:
# make a dictionary of words and the chapters
# where they first appear.
words_chapters = {}

curr_word_chapter = 'I'
for i, div in enumerate(note_text_divs):
    word = div.contents[0].strip('\n')
    # ignore repeated words.
    if word in words_chapters:
        continue
    if i + 1 == len(note_text_divs):
        break
    next_word_chapter = get_chapter_numeral(str(div))
    words_chapters[word] = curr_word_chapter
    curr_word_chapter = next_word_chapter

In [118]:
words_chapters

{'restriñimientos.': 'I',
 'sucedáneos.': 'I',
 'remordimientos': 'I',
 'mentecato.': 'I',
 'veleidoso,': 'I',
 'aturdirse,': 'I',
 'desvencijada,': 'I',
 'apretando': 'I',
 'entablaba': 'I',
 'a lo sumo,': 'I',
 'culebras': 'I',
 'despreocupada,': 'I',
 'se puso colorado': 'I',
 'fulminó': 'I',
 'pátina;': 'I',
 'arrimaron': 'I',
 'carcajadas': 'I',
 'El Magistral le iba a la mano': 'I',
 'Era menester que': 'I',
 'flamantes': 'I',
 'hacer aspavientos.': 'I',
 'pellizco': 'I',
 'calzón': 'I',
 'estrépito': 'I',
 'enaguas;': 'I',
 'disimulaba': 'II',
 'asperezas,': 'II',
 'arrugado': 'II',
 'arcipreste,': 'II',
 'mundanas:': 'II',
 'castidad': 'II',
 'estragos.': 'II',
 'tresillo': 'II',
 'conciliábulos': 'II',
 'alientos': 'II',
 'suelta la sin hueso': 'II',
 'verdores': 'II',
 'boquirroto': 'II',
 'instintos anafrodíticos,': 'II',
 'graneado!': 'II',
 'gallardía': 'II',
 'discurrió': 'II',
 'tacha': 'II',
 'acecho,': 'II',
 'carcajada': 'II',
 'simonía': 'II',
 'jaqueca?': 'II',
 'hi

### Get the kindle location info for the annotated words

In [122]:
def get_kindle_location(div_string):
    kindle_loc_re = r'Location [0-9]+'
    kloc_search = re.search(kindle_loc_re, div_string)
    assert kloc_search is not None
    start, end = kloc_search.span()
    return div_string[start:end]

In [124]:
words_locations = {}

curr_word_location = get_kindle_location(str(soup.find("h3", class_="noteHeading")))
for i, div in enumerate(note_text_divs):
    word = div.contents[0].strip('\n')
    # ignore repeated words.
    if word in words_locations:
        continue
    if i + 1 == len(note_text_divs):
        break
    next_word_location = get_kindle_location(str(div))
    words_locations[word] = curr_word_location
    curr_word_location = next_word_location

In [125]:
words_locations

{'restriñimientos.': 'Location 591',
 'sucedáneos.': 'Location 598',
 'remordimientos': 'Location 602',
 'mentecato.': 'Location 609',
 'veleidoso,': 'Location 611',
 'aturdirse,': 'Location 611',
 'desvencijada,': 'Location 626',
 'apretando': 'Location 631',
 'entablaba': 'Location 638',
 'a lo sumo,': 'Location 640',
 'culebras': 'Location 651',
 'despreocupada,': 'Location 652',
 'se puso colorado': 'Location 663',
 'fulminó': 'Location 681',
 'pátina;': 'Location 684',
 'arrimaron': 'Location 692',
 'carcajadas': 'Location 692',
 'El Magistral le iba a la mano': 'Location 709',
 'Era menester que': 'Location 716',
 'flamantes': 'Location 734',
 'hacer aspavientos.': 'Location 734',
 'pellizco': 'Location 741',
 'calzón': 'Location 746',
 'estrépito': 'Location 754',
 'enaguas;': 'Location 755',
 'disimulaba': 'Location 786',
 'asperezas,': 'Location 786',
 'arrugado': 'Location 796',
 'arcipreste,': 'Location 804',
 'mundanas:': 'Location 808',
 'castidad': 'Location 811',
 'estra

In [171]:
word_chapter_location = []
# due to HTML the first word is a special case
curr_word_location = get_kindle_location(str(soup.find("h3", class_="noteHeading")))
curr_word_chapter = 'I'
for i, div in enumerate(note_text_divs):
    word = div.contents[0].strip('\n')
    # ignore repeated words.
    word_chapter_location.append([word, curr_word_chapter, curr_word_location])
    if i + 1 == len(note_text_divs):  # break if we're at the end of the HTML
        break
    next_word_location = get_kindle_location(str(div))
    next_word_chapter = get_chapter_numeral(str(div))
    curr_word_location = next_word_location
    curr_word_chapter = next_word_chapter

In [172]:
word_chapter_location

[['restriñimientos.', 'I', 'Location 591'],
 ['sucedáneos.', 'I', 'Location 598'],
 ['remordimientos', 'I', 'Location 602'],
 ['mentecato.', 'I', 'Location 609'],
 ['veleidoso,', 'I', 'Location 611'],
 ['aturdirse,', 'I', 'Location 611'],
 ['desvencijada,', 'I', 'Location 626'],
 ['apretando', 'I', 'Location 631'],
 ['entablaba', 'I', 'Location 638'],
 ['a lo sumo,', 'I', 'Location 640'],
 ['culebras', 'I', 'Location 651'],
 ['despreocupada,', 'I', 'Location 652'],
 ['se puso colorado', 'I', 'Location 663'],
 ['fulminó', 'I', 'Location 681'],
 ['pátina;', 'I', 'Location 684'],
 ['arrimaron', 'I', 'Location 692'],
 ['carcajadas', 'I', 'Location 692'],
 ['El Magistral le iba a la mano', 'I', 'Location 709'],
 ['Era menester que', 'I', 'Location 716'],
 ['flamantes', 'I', 'Location 734'],
 ['hacer aspavientos.', 'I', 'Location 734'],
 ['pellizco', 'I', 'Location 741'],
 ['calzón', 'I', 'Location 746'],
 ['estrépito', 'I', 'Location 754'],
 ['enaguas;', 'I', 'Location 755'],
 ['disimulaba'

### Create a `.tsv` file with the word and chapter info.

In [173]:
# remember to open these things with newline=''
with open("words-chapters-locations.tsv", 'w', newline='', encoding='utf-8') as tsv_file:
    writer = csv.writer(tsv_file, delimiter='\t', dialect='excel-tab')
    first_row = ['word', 'chapter', 'location']
    writer.writerow(first_row)
    for sublist in word_chapter_location:
        writer.writerow(sublist)

## Getting the word contexts
You need the first part of _La Regenta_ as a .txt file. Get it from Project Guttenburg.

For each highlighted word, get 300 characters of context in front of it and behind it.

In [174]:
with open('regenta-tomo-I.txt', encoding='utf-8') as f:
    regenta_text = f.read()

In [175]:
def get_context(word, text, n=300):
    """Find WORD in the TEXT with N characters of context.
    
    Get ~300 characters before WORD and ~ 300 characters after it.
    This will probably break if the word comes right before the
    beginning or end of the TEXT. It is also going to find the
    first instance of the word.
    """
    word_idx = text.find(word)
    context = text[word_idx - n : word_idx + n]
    return context

In [176]:
with open('words-chapters-locations.tsv', encoding='utf-8') as testfile:
    testread = testfile.read()

testread

'word\tchapter\tlocation\nrestriñimientos.\tI\tLocation 591\nsucedáneos.\tI\tLocation 598\nremordimientos\tI\tLocation 602\nmentecato.\tI\tLocation 609\nveleidoso,\tI\tLocation 611\naturdirse,\tI\tLocation 611\ndesvencijada,\tI\tLocation 626\napretando\tI\tLocation 631\nentablaba\tI\tLocation 638\na lo sumo,\tI\tLocation 640\nculebras\tI\tLocation 651\ndespreocupada,\tI\tLocation 652\nse puso colorado\tI\tLocation 663\nfulminó\tI\tLocation 681\npátina;\tI\tLocation 684\narrimaron\tI\tLocation 692\ncarcajadas\tI\tLocation 692\nEl Magistral le iba a la mano\tI\tLocation 709\nEra menester que\tI\tLocation 716\nflamantes\tI\tLocation 734\nhacer aspavientos.\tI\tLocation 734\npellizco\tI\tLocation 741\ncalzón\tI\tLocation 746\nestrépito\tI\tLocation 754\nenaguas;\tI\tLocation 755\ndisimulaba\tII\tLocation 786\nasperezas,\tII\tLocation 786\narrugado\tII\tLocation 796\narcipreste,\tII\tLocation 804\nmundanas:\tII\tLocation 808\ncastidad\tII\tLocation 811\nestragos.\tII\tLocation 820\ntresillo

In [185]:
words_context_chapter_location = []

# remember to open these things with newline=''
with open('words-chapters-locations.tsv', newline='', encoding='utf-8') as tsvfile:
    tsv_reader = csv.reader(tsvfile, delimiter='\t', dialect='excel-tab')
    first_row = next(tsv_reader)
    for row in tsv_reader:
        word = row[0]
        chapter = row[1]
        location = row[2]
        context = get_context(word, regenta_text, 300)
        words_context_chapter_location.append([word, context, chapter, location])

words_context_chapter_location

[['restriñimientos.',
  'omo el boj de su huerto. Tenía la boca muy grande, y al sonreír con propósito de agradar, los labios iban de oreja a oreja. No se sabe por qué entonces era cuando mejor se conocía que Bermúdez no se quejaba de vicio al quejarse del pícaro estómago, de digestiones difíciles y sobre todo de perpetuos restriñimientos. Era una sonrisa llena de arrugas, que equivalía a una mueca provocada por un dolor intestinal, aquella con que Bermúdez quería pasar por el hombre más espiritual de Vetusta, y el más capaz de comprender una pasión profunda y alambicada. Pues debe advertirse que sus lecturas serias d',
  'I',
  'Location 591'],
 ['sucedáneos.',
  'no fuera. Sin embargo, pocas veces quitaba la gasa del sombrero porque se tenía por pariente de toda la nobleza vetustense, y en cuanto moría un aristócrata estaba de pésame. Allá, en el fondo de su alma, se creía nacido para el amor, y su pasión por la arqueología era un sentimiento de la clase de sucedáneos. Al ver en las 

### Creating a `.tsv` of Word, its context, its chapter, and its Kindle location.

In [186]:
with open('words-context-chapter-location.tsv', 'w', newline='', encoding='utf-8') as tsvfile:
    tsvwriter = csv.writer(tsvfile, delimiter='\t', dialect='excel-tab')
    first_row = ['word', 'context', 'chapter', 'location']
    tsvwriter.writerow(first_row)
    for sublist in words_context_chapter_location:
        tsvwriter.writerow(sublist)


### Inject HTML tags
So that the words will be highlighted in context on the Anki card. The injected html makes the word <span style="color: rgb(0, 0, 255);">blue in its context.</span>. Injection is done via concatenation so... watch out.


In [192]:
format_tag = '<span style="color: rgb(0, 0, 255);">{}</span>'
format_tag.format('hey')

'<span style="color: rgb(0, 0, 255);">hey</span>'

In [201]:
def style_context(word, context_string, format_tag):
    """Style WORD in CONTEXT_STRING by replacing it with FORMAT_TAG

    This uses concatenation so... watch out.
    """
    before, _, after = context_string.partition(word)
    formatted_word = format_tag.format(word)
    return before + formatted_word + after
    

In [202]:
# an example
style_context('remordimientos', words_context_chapter_location[2][0], format_tag)

'<span style="color: rgb(0, 0, 255);">remordimientos</span>'

In [204]:
format_tag_str = '<span style="color: rgb(0, 0, 255);">{}</span>'

styled_wccl = []
for substring in words_context_chapter_location:
    word = substring[0]
    context = substring[1]
    chapter, location = substring[2], substring[3]
    styled_context = style_context(word, context, format_tag_str)
    styled_wccl.append([word, styled_context, chapter, location])

In [205]:
styled_wccl

[['restriñimientos.',
  'omo el boj de su huerto. Tenía la boca muy grande, y al sonreír con propósito de agradar, los labios iban de oreja a oreja. No se sabe por qué entonces era cuando mejor se conocía que Bermúdez no se quejaba de vicio al quejarse del pícaro estómago, de digestiones difíciles y sobre todo de perpetuos <span style="color: rgb(0, 0, 255);">restriñimientos.</span> Era una sonrisa llena de arrugas, que equivalía a una mueca provocada por un dolor intestinal, aquella con que Bermúdez quería pasar por el hombre más espiritual de Vetusta, y el más capaz de comprender una pasión profunda y alambicada. Pues debe advertirse que sus lecturas serias d',
  'I',
  'Location 591'],
 ['sucedáneos.',
  'no fuera. Sin embargo, pocas veces quitaba la gasa del sombrero porque se tenía por pariente de toda la nobleza vetustense, y en cuanto moría un aristócrata estaba de pésame. Allá, en el fondo de su alma, se creía nacido para el amor, y su pasión por la arqueología era un sentimien

### Creating a pipe-separated file.
Because I'm going to inject HTML formatting when I make the Anki flashcard, and I need a weird delimeter. Like the pipe.

Needs the variable `styled_wccl` from above.

In [207]:
with open('styled_wccl.csv', 'w', newline='', encoding='utf-8') as psvfile:
    # idk if the dialect matters here.
    psvwriter = csv.writer(psvfile, delimiter='|', dialect='excel')
    first_row = ['word', 'styled-context', 'chapter', 'location']
    psvwriter.writerow(first_row)
    for sublist in styled_wccl:
        psvwriter.writerow(sublist)